# Superhero (and Supervillain) Name Generator

---

[Superhero Names Dataset](https://github.com/am1tyadav/superhero)

## Task 2

1. Import the data
2. Create a tokenizer
3. Char to index and Index to char dictionaries

In [3]:
!git clone https://github.com/am1tyadav/superhero

fatal: destination path 'superhero' already exists and is not an empty directory.


In [4]:
with open('superhero/superheroes.txt', 'r') as f:
  data = f.read()

data[:100]

'jumpa\t\ndoctor fate\t\nstarlight\t\nisildur\t\nlasher\t\nvarvara\t\nthe target\t\naxel\t\nbattra\t\nchangeling\t\npyrrh'

In [5]:
import tensorflow as tf
print(tf.__version__)

2.15.0


In [6]:
tokenizer = tf.keras.preprocessing.text.Tokenizer(
    filters='!"#$%&()*+,-./:;<=>?@[\\]^_`{|}~',
    split='\n',
)

In [7]:
tokenizer.fit_on_texts(data)

In [8]:
char_to_index = tokenizer.word_index
index_to_char = dict((v,k) for k, v in char_to_index.items())

print(index_to_char)

{1: '\t', 2: 'a', 3: 'e', 4: 'r', 5: 'o', 6: 'n', 7: 'i', 8: ' ', 9: 't', 10: 's', 11: 'l', 12: 'm', 13: 'h', 14: 'd', 15: 'c', 16: 'u', 17: 'g', 18: 'k', 19: 'b', 20: 'p', 21: 'y', 22: 'w', 23: 'f', 24: 'v', 25: 'j', 26: 'z', 27: 'x', 28: 'q'}


## Task 3

1. Converting between names and sequences

In [9]:
names = data.splitlines()
names[:10]

['jumpa\t',
 'doctor fate\t',
 'starlight\t',
 'isildur\t',
 'lasher\t',
 'varvara\t',
 'the target\t',
 'axel\t',
 'battra\t',
 'changeling\t']

In [10]:
tokenizer.texts_to_sequences(names[0])

[[25], [16], [12], [20], [2], [1]]

In [11]:
def name_to_seq(name):
  return [tokenizer.texts_to_sequences(c)[0][0] for c in name]

In [12]:
name_to_seq(names[0])

[25, 16, 12, 20, 2, 1]

In [13]:
def seq_to_name(seq):
  return ''.join(index_to_char[i] for i in seq if i!=0)

In [14]:
seq_to_name([25, 16, 12, 20, 2, 1])

'jumpa\t'

## Task 4

1. Creating sequences
2. Padding all sequences

In [15]:
sequences = []

for name in names:
  seq = name_to_seq(name)
  if len(seq)>=2:
    sequences += [seq[:i] for i in range(2, len(seq) + 1)]

In [16]:
sequences[:10]

[[25, 16],
 [25, 16, 12],
 [25, 16, 12, 20],
 [25, 16, 12, 20, 2],
 [25, 16, 12, 20, 2, 1],
 [14, 5],
 [14, 5, 15],
 [14, 5, 15, 9],
 [14, 5, 15, 9, 5],
 [14, 5, 15, 9, 5, 4]]

In [18]:
max_len = max((len(x) for x in sequences))
print(max_len)

33


In [21]:
padded_sequences = tf.keras.preprocessing.sequence.pad_sequences(
    sequences, padding = 'pre',
    maxlen = max_len
)
print(padded_sequences[0])

[ 0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0
  0  0  0  0  0  0  0 25 16]


In [22]:
padded_sequences.shape

(88279, 33)

## Task 5: Creating Training and Validation Sets

1. Creating training and validation sets

In [23]:
x, y = padded_sequences[:, :-1], padded_sequences[:, -1]
print(x.shape, y.shape)

(88279, 32) (88279,)


In [26]:
from sklearn.model_selection import train_test_split

x_train, x_test, y_train, y_test = train_test_split(x, y)

print(x_train.shape, y_train.shape)
print(x_test.shape, y_test.shape)

(66209, 32) (66209,)
(22070, 32) (22070,)


In [28]:
print(x_train[0])
print(y_train[0])

[ 0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0
  0  0  0  0  0  0  0 13]
5


In [29]:
num_char = len(char_to_index.keys()) + 1
num_char

29

## Task 6: Creating the Model

In [33]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, Conv1D, MaxPool1D, LSTM, Dense, Bidirectional

model = Sequential([
    Embedding(num_char, 8, input_length = max_len -1),
    Conv1D(64, 5, strides = 1, padding = 'causal', activation = 'tanh'),
    MaxPool1D(2),
    LSTM(32),
    Dense(num_char, activation = 'softmax')
])

model.compile(
    loss = 'sparse_categorical_crossentropy',
    optimizer = 'adam',
    metrics = ['accuracy']
)

model.summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_2 (Embedding)     (None, 32, 8)             232       
                                                                 
 conv1d_2 (Conv1D)           (None, 32, 64)            2624      
                                                                 
 max_pooling1d_1 (MaxPoolin  (None, 16, 64)            0         
 g1D)                                                            
                                                                 
 lstm_1 (LSTM)               (None, 32)                12416     
                                                                 
 dense_1 (Dense)             (None, 29)                957       
                                                                 
Total params: 16229 (63.39 KB)
Trainable params: 16229 (63.39 KB)
Non-trainable params: 0 (0.00 Byte)
__________________

## Task 7: Training the Model

In [34]:
model.fit(x_train, y_train, validation_data = (x_test, y_test), epochs = 50, verbose = 2, batch_size = 32,
          callbacks = [
              tf.keras.callbacks.EarlyStopping(monitor = 'val_accuracy', patience = 3)
          ]
      )

Epoch 1/50
2070/2070 - 17s - loss: 2.7454 - accuracy: 0.1891 - val_loss: 2.5744 - val_accuracy: 0.2246 - 17s/epoch - 8ms/step
Epoch 2/50
2070/2070 - 12s - loss: 2.5224 - accuracy: 0.2420 - val_loss: 2.4904 - val_accuracy: 0.2476 - 12s/epoch - 6ms/step
Epoch 3/50
2070/2070 - 11s - loss: 2.4530 - accuracy: 0.2589 - val_loss: 2.4464 - val_accuracy: 0.2643 - 11s/epoch - 5ms/step
Epoch 4/50
2070/2070 - 11s - loss: 2.4056 - accuracy: 0.2715 - val_loss: 2.4094 - val_accuracy: 0.2719 - 11s/epoch - 5ms/step
Epoch 5/50
2070/2070 - 11s - loss: 2.3700 - accuracy: 0.2805 - val_loss: 2.3906 - val_accuracy: 0.2809 - 11s/epoch - 5ms/step
Epoch 6/50
2070/2070 - 12s - loss: 2.3405 - accuracy: 0.2882 - val_loss: 2.3630 - val_accuracy: 0.2858 - 12s/epoch - 6ms/step
Epoch 7/50
2070/2070 - 11s - loss: 2.3128 - accuracy: 0.2966 - val_loss: 2.3431 - val_accuracy: 0.2932 - 11s/epoch - 5ms/step
Epoch 8/50
2070/2070 - 11s - loss: 2.2866 - accuracy: 0.3066 - val_loss: 2.3228 - val_accuracy: 0.3023 - 11s/epoch - 5

## Task 8: Generate Names!

In [44]:
def generate_names(seed):
  for i in range(0, 40):
    seq = name_to_seq(seed)
    padded = tf.keras.preprocessing.sequence.pad_sequences([seq], padding = 'pre', maxlen = max_len - 1,
                                                           truncating = 'pre')
    pred = model.predict(padded)[0]
    pred_char = index_to_char[tf.argmax(pred).numpy()]
    seed += pred_char

    if pred_char == '\t':
      break
  print(seed)

In [51]:
generate_names('a')

1/1 [==============================] - 0s 18ms/step
artemis	
